# Regression Data

In [69]:
import os 
import pandas as pd
import numpy as np
import h5py
from numpy import nan
from datetime import datetime, timedelta
from time import mktime
from fractions import Fraction
import os.path
pd.set_option('display.max_columns', 500)

In [70]:
def save_or_load_from_checkpoint(checkpoint_name):
    if os.path.isfile(checkpoint_name):
        return pd.read_hdf(checkpoint_name,'table')
    df.to_hdf(checkpoint_name, 'table', mode='w', append=True, complevel=9, complib='zlib', index=False)
    return df

In [71]:
df = save_or_load_from_checkpoint('./checkpoints/data_2.h5')

In [72]:
def get_value(row):
    value = np.nan
    if pd.notna(row['rampFinalValue']):
        value = int(row['rampFinalValue'])
    elif pd.notna(row['temperature']):
        value = int(row['temperature'])
    elif str(row['entryType']) == 'Shades' and str(row['eventType']) == 'SceneChangedEvent':
        base_str = str(row['device_id_name'])
        split_str = base_str.split(' ')
        first_str = split_str[0][-3:]
        is_closed = True if split_str[1] == 'Closed' else False
        if first_str[-1].isdigit():
            frac = float(Fraction(first_str))
            if is_closed:
                frac = 1 - frac
            value = frac
        elif is_closed:
            value = 0
        else:
            value = 1
    return value

In [73]:
df['value'] = df.apply(get_value, axis=1)

In [74]:
def get_event_type(row): 
    event_type = np.nan
    if pd.notna(row['value']):
        event_type = str(row['entryType'])
    return event_type

In [75]:
df['regression_value_type'] = df.apply(get_event_type, axis=1)

In [76]:
regression_df = df[['device_id_name', 'regression_value_type', 'value', 'unix_time']]

In [67]:
regression_df = save_or_load_from_checkpoint('./checkpoints/regression_data.h5')

In [77]:
regression_df

,device_id_name,regression_value_type,value,unix_time
1,74Night,NaN,NaN,1.502438e+09
3,208Basement Mudroom Door,NaN,NaN,1.502438e+09
4,77Lock Basement Mudroom Door,NaN,NaN,1.502438e+09
13,118Sconces,Lights,19275.0,1.502438e+09
14,122Chandelier,Lights,0.0,1.502438e+09
15,120Overhead,Lights,0.0,1.502438e+09
16,118Sconces,Lights,19275.0,1.502438e+09
18,118Sconces,NaN,NaN,1.502438e+09
19,194Master Bed,NaN,NaN,1.502439e+09
20,194Master Bed,NaN,NaN,1.502439e+09
